In [1]:
import scipy as sci
import numpy as np
from qoc.standard import (
                          get_annihilation_operator,
                          get_creation_operator,
                          )
import sys
from qoc.standard.functions.expm_manual import expm_pade
from scipy.sparse.linalg import expm_multiply,expm
from scipy.sparse import dia_matrix,bsr_matrix
from qoc.core.common import expm_frechet
%load_ext line_profiler

def pade(state):
    HILBERT_SIZE=1000
    ANNIHILATION_OPERATOR = get_annihilation_operator(HILBERT_SIZE)
    CREATION_OPERATOR = get_creation_operator(HILBERT_SIZE)
    H0=np.matmul(CREATION_OPERATOR,ANNIHILATION_OPERATOR)+0.1*(ANNIHILATION_OPERATOR+CREATION_OPERATOR)
    A=-1j*0.1*H0
    propagator=expm_pade(A)

    return np.matmul(propagator,state)

def krylov(state):
    HILBERT_SIZE = 1000
    ANNIHILATION_OPERATOR = get_annihilation_operator(HILBERT_SIZE)
    CREATION_OPERATOR = get_creation_operator(HILBERT_SIZE)
    H0 = np.matmul(CREATION_OPERATOR, ANNIHILATION_OPERATOR) + 0.1 * (ANNIHILATION_OPERATOR + CREATION_OPERATOR)
    A = -1j * 0.1 * H0
    state=expm_multiply(A,state)
    return state
def sci_expm(state):
    HILBERT_SIZE = 100
    diagnol=np.arange(HILBERT_SIZE)
    up_diagnol=np.sqrt(diagnol)
    low_diagnol=np.sqrt(np.arange(1,HILBERT_SIZE+1))
    data=[low_diagnol,diagnol,up_diagnol]
    offsets=[-1,0,1]
    A=-1j*0.1*dia_matrix((data, offsets),shape=(HILBERT_SIZE,HILBERT_SIZE)).tocsc()
    B=-1j*0.1*dia_matrix(([low_diagnol,up_diagnol], [-1,1]),shape=(HILBERT_SIZE,HILBERT_SIZE)).tocsc()
    state=expm(A)*state
    return state
def krylov_sparse(state):
    HILBERT_SIZE = 3000
    diagnol=np.arange(HILBERT_SIZE)
    up_diagnol=np.sqrt(diagnol)
    low_diagnol=np.sqrt(np.arange(1,HILBERT_SIZE+1))
    data=[low_diagnol,diagnol,up_diagnol]
    offsets=[-1,0,1]
    A=-1j*0.1*dia_matrix((data, offsets),shape=(HILBERT_SIZE,HILBERT_SIZE))
    state=expm_multiply(A,state)
    return state
def derivative_Expansion():
    HILBERT_SIZE = 10000
    diagnol = np.arange(HILBERT_SIZE)
    up_diagnol = np.sqrt(diagnol)
    low_diagnol = np.sqrt(np.arange(1, HILBERT_SIZE + 1))
    data = [low_diagnol, diagnol, up_diagnol]
    offsets = [-1, 0, 1]
    A = dia_matrix((data, offsets), shape=(HILBERT_SIZE, HILBERT_SIZE)).todense()
    B = dia_matrix(([low_diagnol, up_diagnol], [-1, 1]), shape=(HILBERT_SIZE, HILBERT_SIZE)).todense()
    c = np.block([[A, B], [np.zeros_like(A), A]])
    c=bsr_matrix(c).tocsc()
    return expm(c)
def frechet():
    HILBERT_SIZE = 1000
    ANNIHILATION_OPERATOR = get_annihilation_operator(HILBERT_SIZE)
    CREATION_OPERATOR = get_creation_operator(HILBERT_SIZE)
    H0 = np.matmul(CREATION_OPERATOR, ANNIHILATION_OPERATOR) + 0.1 * (ANNIHILATION_OPERATOR + CREATION_OPERATOR)
    A = -1j * 0.1 * H0
    E=  -1j * 0.1 * (ANNIHILATION_OPERATOR + CREATION_OPERATOR)
    expm_frechet(A,E)
dim=1000
state=np.sqrt(1/dim)*np.zeros(dim)


np.set_printoptions(threshold=sys.maxsize)
#a=pade(state)
#b=sci_expm(state)



In [5]:
%timeit -r 1 -n 1  derivative_Expansion()

14min 22s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [12]:
%timeit -r 10 -n 10  krylov(state)

39.6 ms ± 2.33 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [16]:
%timeit -r 1 -n 3  pade(state)

824 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 3 loops each)


In [2]:
%timeit -r 1 -n 1  frechet()

2.51 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
